## Imports

In [1]:
import os
import pandas as pd
import backtrader as bt
import requests
import json
import matplotlib

## Data Fetching

### Token Metrics API

In [2]:
url = "https://api.tokenmetrics.com/v1/trading-indicator"
querystring = {"tokens": "3375", "limit": "1830"}
tm_api_key = os.environ.get('tokenMetrics_API')
header = {"api_key": tm_api_key}
response = requests.request("GET", url, headers=header, params=querystring)


In [3]:
data = json.loads(response.text)


In [4]:
df_tm = pd.DataFrame.from_dict(data['data']).set_index(
    'DATE').sort_index(ascending=True)


In [5]:
df_tm

,TOKEN_ID,NAME,SYMBOL,CLOSE,VOLUME,SIGNAL,LAST_SIGNAL,HOLDING_CUMULATIVE_ROI,STRATEGY_CUMULATIVE_ROI,EPOCH
DATE,,,,,,,,,,
2018-04-12,3375,Bitcoin,BTC,7836.396973,1.586992e+09,0,-1,-0.288031,-0.192228,1523491200
2018-04-13,3375,Bitcoin,BTC,7845.752930,4.980343e+09,0,-1,-0.287181,-0.192228,1523577600
2018-04-14,3375,Bitcoin,BTC,7897.719727,3.910298e+09,0,-1,-0.282459,-0.192228,1523664000
2018-04-15,3375,Bitcoin,BTC,8258.067383,2.294694e+09,0,-1,-0.249720,-0.192228,1523750400
2018-04-16,3375,Bitcoin,BTC,8030.999023,2.350691e+09,0,-1,-0.270350,-0.192228,1523836800
...,...,...,...,...,...,...,...,...,...,...
2023-05-26,3375,Bitcoin,BTC,26473.000000,1.138485e+10,0,-1,1.405183,4.749552,1685059200
2023-05-27,3375,Bitcoin,BTC,26684.000000,1.117483e+10,0,-1,1.424353,4.749552,1685145600
2023-05-28,3375,Bitcoin,BTC,26806.000000,6.834437e+09,0,-1,1.435437,4.749552,1685232000


## Data Preprocessing

### OHLCV

In [6]:
df_tm = df_tm[['CLOSE', 'VOLUME']]


In [7]:
df_tm['open'] = df_tm['high'] = df_tm['low'] = df_tm['CLOSE']


/var/folders/rk/k5c7pd0n12jbjgcrrq2w32980000gn/T/ipykernel_26286/3096381224.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tm['open'] = df_tm['high'] = df_tm['low'] = df_tm['CLOSE']
/var/folders/rk/k5c7pd0n12jbjgcrrq2w32980000gn/T/ipykernel_26286/3096381224.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tm['open'] = df_tm['high'] = df_tm['low'] = df_tm['CLOSE']
/var/folders/rk/k5c7pd0n12jbjgcrrq2w32980000gn/T/ipykernel_26286/3096381224.py:1: SettingWithCopyWarning: 
A value is trying to be s

In [8]:
df_tm = df_tm.iloc[:,[2,3,4,0,1]]

In [9]:
df_tm.columns = ['Open', 'High', 'Low', 'Close', 'Volume']

In [10]:
df_tm.index = pd.to_datetime(df_tm.index)

In [11]:
df_tm

,Open,High,Low,Close,Volume
DATE,,,,,
2018-04-12,7836.396973,7836.396973,7836.396973,7836.396973,1.586992e+09
2018-04-13,7845.752930,7845.752930,7845.752930,7845.752930,4.980343e+09
2018-04-14,7897.719727,7897.719727,7897.719727,7897.719727,3.910298e+09
2018-04-15,8258.067383,8258.067383,8258.067383,8258.067383,2.294694e+09
2018-04-16,8030.999023,8030.999023,8030.999023,8030.999023,2.350691e+09
...,...,...,...,...,...
2023-05-26,26473.000000,26473.000000,26473.000000,26473.000000,1.138485e+10
2023-05-27,26684.000000,26684.000000,26684.000000,26684.000000,1.117483e+10
2023-05-28,26806.000000,26806.000000,26806.000000,26806.000000,6.834437e+09


In [18]:
df_tm.to_csv('data/BTC-USD_tm_.csv')

---

# Backtesting

In [12]:
import backtrader.analyzers as btanalyzers
from backtrader.feeds import GenericCSVData
import datetime


## Strategies

In [13]:
class trendStrategy(bt.Strategy):
    '''
    Used to create new customized strategy by extending the base bt.Strategy Class
    '''

    # tuple to store parameter values. access them using self.p.p-name
    params = (
        ('stopwin', 0.3),
        ('stoploss', 0.05),
        ('mid_sma', 50)
    )

    # function which prints any activity to the console.
    def log(self, txt, dt=None):
        # we have referenced above datetime here as datetime
        date_str = self.data.datetime.date().isoformat()
        txt = 'Date: {}, {}'.format(date_str, txt)
        print(txt)

    def __init__(self):
        # store the column 'close' to instantly get the closing price by using self.dataclose[0]
        # could either do this or self.data.col-name[0]
        self.dataclose = self.datas[0].close
        # to keep track of any pending orders, last buy-price and commission
        self.order = None
        # Add MovingAverageSimple indicators
        self.price_sma_mid = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.mid_sma, plot=True)

    def notify_order(self, order):
        '''
        invoked after self.buy() or self.sell()
        To tell user about the order summary and to set stoploss and stopwin for each individual order.
        '''

        if order.status in [order.Submitted, order.Accepted]:
            # If Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            # Check if an order has been completed
            if order.isbuy():
                if self.position.size > 0:  # If we've taken a long position
                    # Take note of buy-price, commission, Max/Min Sell price
                    self.buyprice = order.executed.price
                    self.buycomm = order.executed.comm
                    self.max_sell_price = order.executed.price * \
                        (1+self.params.stopwin)  # 30% stop-profit
                    self.min_sell_price = order.executed.price * \
                        (1-self.params.stoploss)  # 5% stop-loss
                    self.log(
                        'Long Position Taken. Price: {}, Cost: {}, Commission: {}, Max Sell: {}, Min Sell: {}, Position Size: {}'.format(
                            order.executed.price,
                            order.executed.value,
                            order.executed.comm,
                            self.max_sell_price,
                            self.min_sell_price,
                            self.position.size)
                    )

                if self.position.size == 0:  # If cleared a short position
                    self.log('Short Position Cleared. Price: {}, Cost: {}, Commission: {}'.format(
                        order.executed.price,
                        order.executed.value,
                        order.executed.comm
                    ))

            elif order.issell():
                if self.position.size < 0:  # If taken a short position
                    self.min_sell_price = order.executed.price * \
                        (1-self.params.stopwin)  # 30% stop-profit
                    self.max_sell_price = order.executed.price * \
                        (1+self.params.stoploss)  # 5% stop-loss
                    self.log('Short Position Taken. Price: {}, Cost: {}, Commission: {}, Max Sell: {}, Min Sell: {}, Position Size: {}'.format(
                        order.executed.price,
                        order.executed.value,
                        order.executed.comm,
                        self.max_sell_price,
                        self.min_sell_price,
                        self.position.size
                    ))
                if self.position.size == 0:  # If cleared a long position
                    self.log('Long Position Cleared. Price: {}, Cost: {}, Commission: {}, Position Size: {}'.format(
                        order.executed.price,
                        order.executed.value,
                        order.executed.comm,
                        self.position.size))

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        '''
        If order is completed or for some reason canceled/rejected you close the order here.
        '''
        self.order = None

    def notify_trade(self, trade):
        '''
        trade refers to a buy/sell pair
        invoked after a buy or sell order is executed i.e. completed.
        '''

        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        '''
        this method is called whenever each new bar (next row in data) is encountered
        '''

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
        
        # checking if we've already taken a position (i.e. self.position.size = 0) can't take another position if you've already taken a position
        if not self.position:
            if self.dataclose[0] > self.price_sma_mid[0]: # if current close price crosses above sma(50), go long
                self.log('Long Position Entry Signalled. Price: {} Position Size: {}'.format(
                    self.dataclose[0], self.position.size
                ))
                self.order = self.buy()

            elif self.dataclose[0] < self.price_sma_mid[0]: # if current close crosses below sma(50), go short
                self.log('Short Position Entry Signalled. Price: {}, Position Size: {}'.format(
                    self.dataclose[0], self.position.size
                ))
                self.order = self.sell()

        else:  # Clear Position

            if self.position.size > 0:  # If long

                if self.dataclose[0] >= self.max_sell_price: # stopwin is reached
                    self.log('Long Position Clear Signal. Stopwin Reached. Price: {} Position Size: {}'.format(
                        self.dataclose[0], self.position.size))

                    self.order = self.sell()

                elif self.dataclose[0] <= self.min_sell_price: # stoploss is reached
                    self.log('Long Position Clear Signal. Stoploss Reached. Price: {} Position Size: {}'.format(
                        self.dataclose[0], self.position.size))

                    self.order = self.sell()

            elif self.position.size < 0:  # if short

                if self.dataclose[0] >= self.max_sell_price:
                    self.log('Short Position Clear Signal. Stoploss reached. Price: {}, Position Size: {}'.format(
                        self.dataclose[0], self.position.size
                    ))
                    self.order = self.buy()

                elif self.dataclose[0] <= self.min_sell_price:
                    self.log('Short Position Clear Signal. Stopwin reached. Price: {}, Position Size: {}'.format(
                        self.dataclose[0], self.position.size
                    ))
                    self.order = self.buy()

    def stop(self):
        '''
        executes after strategy is done executing i.e. at the very end
        '''
        drawdown = self.analyzers.myDrawdown.get_analysis()
        print('max drawdown: {}'.format(drawdown.max.drawdown))


In [14]:
class EMATrendStrategy_1(bt.Strategy):
    '''
    Used to create new customized strategy by extending the base bt.Strategy Class
    '''

    # tuple to store parameter values. access them using self.p.p-name
    params = (
        ('stopwin', 0.3),
        ('stoploss', 0.05),
        ('fast_ema', 36),
        ('slow_ema', 60),
        ('sizer', 0.2)
    )

    # function which prints any activity to the console.
    def log(self, txt, dt=None):
        # we have referenced above datetime here as datetime
        date_str = self.data.datetime.datetime().isoformat()
        txt = 'Date: {}, {}'.format(date_str, txt)
        print(txt)

    def __init__(self):
        # store the column 'close' to instantly get the closing price by using self.dataclose[0]
        # could either do this or self.data.col-name[0]
        self.dataclose = self.datas[0].close
        # to keep track of any pending orders, last buy-price and commission
        self.order = None
        # Add MovingAverage indicators
        self.price_ema_fast = bt.indicators.ExponentialMovingAverage(
            period=self.params.fast_ema
        )
        self.price_ema_slow = bt.indicators.ExponentialMovingAverage(
            period = self.params.slow_ema
        )

    def notify_order(self, order):
        '''
        invoked after self.buy() or self.sell()
        To tell user about the order summary and to set stoploss and stopwin for each individual order.
        '''

        if order.status in [order.Submitted, order.Accepted]:
            # If Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            # Check if an order has been completed
            if order.isbuy():
                if self.position.size > 0:  # If we've taken a long position
                    # Take note of buy-price, commission, Max/Min Sell price
                    self.buyprice = order.executed.price
                    self.buycomm = order.executed.comm
                    self.max_sell_price = order.executed.price * \
                        (1+self.params.stopwin)  # 30% stop-profit
                    self.min_sell_price = order.executed.price * \
                        (1-self.params.stoploss)  # 5% stop-loss
                    self.log(
                        'Long Position Taken. Price: {}, Cost: {}, Commission: {}, Max Sell: {}, Min Sell: {}, Position Size: {}'.format(
                            order.executed.price,
                            order.executed.value,
                            order.executed.comm,
                            self.max_sell_price,
                            self.min_sell_price,
                            self.position.size)
                    )

                if self.position.size == 0:  # If cleared a short position
                    self.log('Short Position Cleared. Price: {}, Cost: {}, Commission: {}'.format(
                        order.executed.price,
                        order.executed.value,
                        order.executed.comm
                    ))

            elif order.issell():
                if self.position.size < 0:  # If taken a short position
                    self.min_sell_price = order.executed.price * \
                        (1-self.params.stopwin)  # 30% stop-profit
                    self.max_sell_price = order.executed.price * \
                        (1+self.params.stoploss)  # 5% stop-loss
                    self.log('Short Position Taken. Price: {}, Cost: {}, Commission: {}, Max Sell: {}, Min Sell: {}, Position Size: {}'.format(
                        order.executed.price,
                        order.executed.value,
                        order.executed.comm,
                        self.max_sell_price,
                        self.min_sell_price,
                        self.position.size
                    ))
                if self.position.size == 0:  # If cleared a long position
                    self.log('Long Position Cleared. Price: {}, Cost: {}, Commission: {}, Position Size: {}'.format(
                        order.executed.price,
                        order.executed.value,
                        order.executed.comm,
                        self.position.size))

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        '''
        If order is completed or for some reason canceled/rejected you close the order here.
        '''
        self.order = None

    def notify_trade(self, trade):
        '''
        trade refers to a buy/sell pair
        invoked after a buy or sell order is executed i.e. completed.
        '''

        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        '''
        this method is called whenever each new bar (next row in data) is encountered
        '''

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        if not self.position:

            if self.price_ema_fast[0] > self.price_ema_slow[0]:
                self.log('Signal: Long Position Price: {}'.format(
                    self.dataclose[0]))
                self.order = self.buy()
            
            elif self.price_ema_fast[0] < self.price_ema_slow[0]:
                self.log('Signal: Short Position. Price: {}'.format(self.dataclose[0]))
                self.order = self.sell()
                    
                
            # if self.position:
            #     if self.position.size < 0:
            #         self.log('Signal: Short -> Long. Price: {}'.format(self.dataclose[0]))
            #         self.order = self.buy(size=2*self.params.sizer)
                
            # if self.position:
            #     if self.position.size > 0:
            #         self.log('Signal: Long -> Short. Price: {}'.format(self.dataclose[0]))
            #         self.order = self.sell(size=2*self.params.sizer)

# TODO this else will never execute
# restructure based on whether a position is taken

        else:  # Clear Position

            if self.position.size > 0:  # If long

                if self.dataclose[0] >= self.max_sell_price:  # stopwin is reached
                    self.log('Long Position Clear Signal. Stopwin Reached. Price: {} Position Size: {}'.format(
                        self.dataclose[0], self.position.size))

                    self.order = self.sell()

                elif self.dataclose[0] <= self.min_sell_price:  # stoploss is reached
                    self.log('Long Position Clear Signal. Stoploss Reached. Price: {} Position Size: {}'.format(
                        self.dataclose[0], self.position.size))

                    self.order = self.sell()

            elif self.position.size < 0:  # if short

                if self.dataclose[0] >= self.max_sell_price:
                    self.log('Short Position Clear Signal. Stoploss reached. Price: {}, Position Size: {}'.format(
                        self.dataclose[0], self.position.size
                    ))
                    self.order = self.buy()

                elif self.dataclose[0] <= self.min_sell_price:
                    self.log('Short Position Clear Signal. Stopwin reached. Price: {}, Position Size: {}'.format(
                        self.dataclose[0], self.position.size
                    ))
                    self.order = self.buy()

    def stop(self):
        '''
        executes after strategy is done executing i.e. at the very end
        '''
        drawdown = self.analyzers.myDrawdown.get_analysis()
        print('max drawdown: {}'.format(drawdown.max.drawdown))


In [15]:
class EMATrendStrategy_2(bt.Strategy):

    # tuple to store parameter values. access them using self.p.p-name
    params = (
        ('stopwin', 0.3),
        ('stoploss', 0.05),
        ('fast_ema', 36),
        ('slow_ema', 60),
        ('sizer', 0.2)
    )

    # function which prints any activity to the console.
    def log(self, txt, dt=None):
        # we have referenced above datetime here as datetime
        date_str = self.data.datetime.datetime().isoformat()
        txt = 'Date: {}, {}'.format(date_str, txt)
        print(txt)

    def __init__(self):
        # store the column 'close' to instantly get the closing price by using self.dataclose[0]
        # could either do this or self.data.col-name[0]
        self.dataclose = self.datas[0].close
        # to keep track of any pending orders, last buy-price and commission
        self.order = None
        # Add MovingAverage indicators
        self.ema_fast = bt.indicators.ExponentialMovingAverage(
            self.datas[0], period=self.p.fast_ema
        )
        self.ema_slow = bt.indicators.ExponentialMovingAverage(
            self.datas[0], period=self.p.slow_ema
        )

    def notify_order(self, order):
        '''
        invoked after self.buy() or self.sell()
        To tell user about the order summary and to set stoploss and stopwin for each individual order.
        '''

        if order.status in [order.Submitted, order.Accepted]:
            # If Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            # Check if an order has been completed
            if order.isbuy():
                if self.position.size > 0:  # If we've taken a long position
                    # Take note of buy-price, commission, Max/Min Sell price
                    self.buyprice = order.executed.price
                    self.buycomm = order.executed.comm
                    self.max_sell_price = order.executed.price * \
                        (1+self.params.stopwin)  # 30% stop-profit
                    self.min_sell_price = order.executed.price * \
                        (1-self.params.stoploss)  # 5% stop-loss
                    self.log(
                        'Long Position Taken. Price: {}, Cost: {}, Commission: {}, Max Sell: {}, Min Sell: {}, Position Size: {}, Portfolio Value: {}'.format(
                            order.executed.price,
                            order.executed.value,
                            order.executed.comm,
                            self.max_sell_price,
                            self.min_sell_price,
                            self.position.size,
                            self.broker.get_value())
                    )

                if self.position.size == 0:  # If cleared a short position
                    self.log('Short Position Cleared. Price: {}, Cost: {}, Commission: {}, Portfolio Value: {}'.format(
                        order.executed.price,
                        order.executed.value,
                        order.executed.comm,
                        self.broker.get_value()
                    ))

            elif order.issell():
                if self.position.size < 0:  # If taken a short position
                    self.buyprice = order.executed.price
                    self.min_sell_price = order.executed.price * \
                        (1-self.params.stopwin)  # 30% stop-win
                    self.max_sell_price = order.executed.price * \
                        (1+self.params.stoploss)  # 5% stop-loss
                    self.log('Short Position Taken. Price: {}, Cost: {}, Commission: {}, Max Sell: {}, Min Sell: {}, Position Size: {}'.format(
                        order.executed.price,
                        order.executed.value,
                        order.executed.comm,
                        self.max_sell_price,
                        self.min_sell_price,
                        self.position.size
                    ))
                if self.position.size == 0:  # If cleared a long position
                    self.log('Long Position Cleared. Price: {}, Cost: {}, Commission: {}, Position Size: {}'.format(
                        order.executed.price,
                        order.executed.value,
                        order.executed.comm,
                        self.position.size))

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        '''
        If order is completed or for some reason canceled/rejected you close the order here.
        '''
        self.order = None

    def notify_trade(self, trade):
        '''
        trade refers to a buy/sell pair
        invoked after a buy or sell order is executed i.e. completed.
        '''

        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        '''
        this method is called whenever each new bar (next row in data) is encountered
        '''

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        if not self.position:

            if self.ema_fast[0] > self.ema_slow[0]:
                self.order = self.buy()
                self.log('Signal: Long Position. Price: {}'.format(
                    self.dataclose[0]))

            if self.ema_fast[0] < self.ema_slow[0]:
                self.log('Signal: Short Position. Price: {}'.format(
                    self.dataclose[0]))
                self.order = self.sell()

        else:
            if self.position.size > 0:
                if self.dataclose[0] <= self.min_sell_price:   # stoploss is reached
                    self.log('Long Position Clear Signal. Stoploss Reached. Price: {} Position Size: {}'.format(
                        self.dataclose[0], self.position.size))

                    self.order = self.close()
                    # self.order = self.sell(self.position.size)

                elif self.ema_fast[0] > self.ema_slow[0]:  # buy signal when long
                    # the requirement that last order is long, is satisfied by condition self.position.size > 0
                    if self.dataclose[0] > self.buyprice:
                        self.log('Long Position Increase Signal. Price: {} Position Size: {}'.format(
                            self.dataclose[0], self.position.size))
                        self.order = self.buy(
                            size=self.p.sizer)  # buy 0.2 shares

                # TODO maybe also buy a share here
                elif self.ema_fast[0] < self.ema_slow[0]:
                    self.log('Long Position Clear Signal. Downtrend Detected. Price: {} Position Size: {}'.format(
                        self.dataclose[0], self.position.size))
                    self.order = self.close()
                    # self.order = self.sell(self.position.size)

                elif self.dataclose[0] > self.max_sell_price:  # stopwin reached
                    self.log('Short Position Clear Signal. Stopwin Reached. Price: {} Position Size: {}'.format(
                        self.dataclose[0], self.position.size))
                    self.order = self.close()
                    # self.order = self.sell(self.position.size)

            elif self.position.size < 0:
                if self.dataclose[0] > self.max_sell_price:
                    self.log('Short Position Clear Signal. Stoploss Reached. Price: {} Position Size: {}'.format(
                        self.dataclose[0], self.position.size))
                    self.order = self.close()
                    # self.order = self.buy(abs(self.position.size))

                # really make sure that you wanna clear all your short positions.
                elif self.ema_fast[0] > self.ema_slow[0]:
                    self.log('Short Position Clear Signal. Uptrend Detected. Price: {} Position Size: {}'.format(
                        self.dataclose[0], self.position.size))
                    self.order = self.close()
                    # self.order = self.buy(abs(self.position.size))

                elif self.ema_fast[0] < self.ema_slow[0]:  # sell signal when short
                    if self.dataclose[0] < self.buyprice:
                        self.log('Short Position Increase Signal. Price: {} Position Size: {}'.format(
                            self.dataclose[0], self.position.size))
                        # TODO add quantity and add log message
                        self.order = self.sell(size=self.p.sizer)

                elif self.dataclose[0] < self.min_sell_price:  # stopwin reached
                    self.log('Short Position Clear Signal. Stopwin Reached. Price: {} Position Size: {}'.format(
                        self.dataclose[0], self.position.size))
                    self.order = self.close()
                    # self.order = self.buy(abs(self.position.size))

    def stop(self):
        '''
        executes after strategy is done executing i.e. at the very end
        '''
        self.order = self.close()
        self.log('Last Position Close Signal. Price: {} Position Size: {}'.format(
            self.dataclose[0], self.position.size))
        drawdown = self.analyzers.myDrawdown.get_analysis()
        print('max drawdown: {}'.format(drawdown.max.drawdown))


## Datafeeds

In [16]:
df_tm = pd.read_csv('data/BTC-USD_tm.csv', parse_dates=True, index_col=0)


In [17]:
df = df_tm


In [19]:
data = bt.feeds.PandasData(
    dataname=df,
    datetime=None,
    open=0,
    high=1,
    low=2,
    close=3,
    volume=4,
    openinterest=None
)


## Cerebro Instance

Strategy-1

In [20]:
cerebro = bt.Cerebro(stdstats=False)

# add default observers
cerebro.addobserver(bt.observers.Broker, plot=False)
cerebro.addobserver(bt.observers.Trades)
cerebro.addobserver(bt.observers.BuySell)
cerebro.adddata(data)

cerebro.broker.setcash(100000)

cerebro.broker.setcommission(commission=0.001)

# annaul Sharpe Ratio
cerebro.addanalyzer(btanalyzers.SharpeRatio_A,
                    _name='mySharpe', riskfreerate=0.07)

# Drawdown
cerebro.addanalyzer(btanalyzers.DrawDown, _name='myDrawdown')

cerebro.addanalyzer(btanalyzers.TradeAnalyzer, _name='myTradeAnalyzer')


In [21]:
cerebro.addstrategy(trendStrategy)
start_value = cerebro.broker.getvalue()
print('------------------------------------------------')
print('Starting Portfolio Value: {}'.format(start_value))

backtest_result = cerebro.run()

sr = backtest_result[0].analyzers.mySharpe.get_analysis()['sharperatio']
print('Sharpe Ratio: {}'.format(sr))
tt = backtest_result[0].analyzers.getbyname(
    'myTradeAnalyzer').get_analysis()['total']['total']
print('total trades:', tt)
profit_win = backtest_result[0].analyzers.getbyname(
    'myTradeAnalyzer').get_analysis()['won']['pnl']['average']
print('profit/winning trade:', profit_win)
loss_lose = backtest_result[0].analyzers.getbyname(
    'myTradeAnalyzer').get_analysis()['lost']['pnl']['average']
print('loss/losing trade:', loss_lose)
pnl = backtest_result[0].analyzers.getbyname(
    'myTradeAnalyzer').get_analysis()['pnl']['net']['total']
print('PnL:', pnl)
avg_pnl = backtest_result[0].analyzers.getbyname(
    'myTradeAnalyzer').get_analysis()['pnl']['net']['average']
print('Avg PnL/Trade:', avg_pnl)

end_value = cerebro.broker.getvalue()

abs_rtn = (end_value - start_value)/start_value
print('Absolute Returns: {}%'.format(abs_rtn*100))

no_days = len(df.asfreq(freq='D').dropna())
cagr = (end_value/start_value)**(365/no_days) - 1
print('CAGR: {}%'.format(cagr*100))

print('Ending Portfolio Value: {}'.format(end_value))


------------------------------------------------
Starting Portfolio Value: 100000
Date: 2018-05-28, Short Position Entry Signalled. Price: 7111.0688476562, Position Size: 0
Date: 2018-05-29, Short Position Taken. Price: 8055.508227539054, Cost: -8055.508227539054, Commission: 8.055508227539054, Max Sell: 8458.283638916007, Min Sell: 5638.855759277338, Position Size: -1
Date: 2018-11-16, Short Position Clear Signal. Stopwin reached. Price: 5609.9560546875, Position Size: -1
Date: 2018-11-17, Short Position Cleared. Price: 6262.963598632815, Cost: -8055.508227539054, Commission: 6.262963598632815
Date: 2018-11-17, OPERATION PROFIT, GROSS 1792.54, NET 1778.23
Date: 2018-11-17, Short Position Entry Signalled. Price: 5599.19140625, Position Size: 0
Date: 2018-11-18, Short Position Taken. Price: 6230.274267578125, Cost: -6230.274267578125, Commission: 6.230274267578125, Max Sell: 6541.787980957031, Min Sell: 4361.191987304687, Position Size: -1
Date: 2018-11-22, Short Position Clear Signal. 

Strategy-2

In [23]:
cerebro = bt.Cerebro(stdstats=False)

# add default observers
cerebro.addobserver(bt.observers.Broker, plot=False)
cerebro.addobserver(bt.observers.Trades)
cerebro.addobserver(bt.observers.BuySell)
cerebro.adddata(data)

cerebro.broker.setcash(100000)

cerebro.broker.setcommission(commission=0.001)

# annaul Sharpe Ratio
cerebro.addanalyzer(btanalyzers.SharpeRatio_A,
                    _name='mySharpe', riskfreerate=0.07)

# Drawdown
cerebro.addanalyzer(btanalyzers.DrawDown, _name='myDrawdown')

cerebro.addanalyzer(btanalyzers.TradeAnalyzer, _name='myTradeAnalyzer')

cerebro.addstrategy(EMATrendStrategy_1)
start_value = cerebro.broker.getvalue()
print('------------------------------------------------')
print('Starting Portfolio Value: {}'.format(start_value))

backtest_result = cerebro.run()

sr = backtest_result[0].analyzers.mySharpe.get_analysis()['sharperatio']
print('Sharpe Ratio: {}'.format(sr))
tt = backtest_result[0].analyzers.getbyname(
    'myTradeAnalyzer').get_analysis()['total']['total']
print('total trades:', tt)
profit_win = backtest_result[0].analyzers.getbyname(
    'myTradeAnalyzer').get_analysis()['won']['pnl']['average']
print('profit/winning trade:', profit_win)
loss_lose = backtest_result[0].analyzers.getbyname(
    'myTradeAnalyzer').get_analysis()['lost']['pnl']['average']
print('loss/losing trade:', loss_lose)
pnl = backtest_result[0].analyzers.getbyname(
    'myTradeAnalyzer').get_analysis()['pnl']['net']['total']
print('PnL:', pnl)
avg_pnl = backtest_result[0].analyzers.getbyname(
    'myTradeAnalyzer').get_analysis()['pnl']['net']['average']
print('Avg PnL/Trade:', avg_pnl)

end_value = cerebro.broker.getvalue()

abs_rtn = (end_value - start_value)/start_value
print('Absolute Returns: {}%'.format(abs_rtn*100))

no_days = len(df.asfreq(freq='D').dropna())
cagr = (end_value/start_value)**(365/no_days) - 1
print('CAGR: {}%'.format(cagr*100))

print('Ending Portfolio Value: {}'.format(end_value))


------------------------------------------------
Starting Portfolio Value: 100000
Date: 2018-06-07T00:00:00, Signal: Short Position. Price: 7613.77734375
Date: 2018-06-08T00:00:00, Short Position Taken. Price: 7598.290966796865, Cost: -7598.290966796865, Commission: 7.598290966796865, Max Sell: 7978.2055151367085, Min Sell: 5318.803676757805, Position Size: -1
Date: 2018-07-24T00:00:00, Short Position Clear Signal. Stoploss reached. Price: 8422.3740234375, Position Size: -1
Date: 2018-07-25T00:00:00, Short Position Cleared. Price: 6983.244750976545, Cost: -7598.290966796865, Commission: 6.983244750976545
Date: 2018-07-25T00:00:00, OPERATION PROFIT, GROSS 615.05, NET 600.46
Date: 2018-07-25T00:00:00, Signal: Short Position. Price: 8144.5537109375
Date: 2018-07-26T00:00:00, Short Position Taken. Price: 7049.53395996092, Cost: -7049.53395996092, Commission: 7.04953395996092, Max Sell: 7402.010657958966, Min Sell: 4934.673771972643, Position Size: -1
Date: 2018-07-26T00:00:00, Short Positi

Strategy-3

In [24]:
cerebro = bt.Cerebro(stdstats=False)

# add default observers
cerebro.addobserver(bt.observers.Broker, plot=False)
cerebro.addobserver(bt.observers.Trades)
cerebro.addobserver(bt.observers.BuySell)
cerebro.adddata(data)

cerebro.broker.setcash(100000)

cerebro.broker.setcommission(commission=0.001)

# annaul Sharpe Ratio
cerebro.addanalyzer(btanalyzers.SharpeRatio_A,
                    _name='mySharpe', riskfreerate=0.07)

# Drawdown
cerebro.addanalyzer(btanalyzers.DrawDown, _name='myDrawdown')

cerebro.addanalyzer(btanalyzers.TradeAnalyzer, _name='myTradeAnalyzer')


cerebro.addstrategy(EMATrendStrategy_2)
start_value = cerebro.broker.getvalue()
print('------------------------------------------------')
print('Starting Portfolio Value: {}'.format(start_value))

backtest_result = cerebro.run()

sr = backtest_result[0].analyzers.mySharpe.get_analysis()['sharperatio']
print('Sharpe Ratio: {}'.format(sr))
tt = backtest_result[0].analyzers.getbyname(
    'myTradeAnalyzer').get_analysis()['total']['total']
print('total trades:', tt)
profit_win = backtest_result[0].analyzers.getbyname(
    'myTradeAnalyzer').get_analysis()['won']['pnl']['average']
print('profit/winning trade:', profit_win)
loss_lose = backtest_result[0].analyzers.getbyname(
    'myTradeAnalyzer').get_analysis()['lost']['pnl']['average']
print('loss/losing trade:', loss_lose)
pnl = backtest_result[0].analyzers.getbyname(
    'myTradeAnalyzer').get_analysis()['pnl']['net']['total']
print('PnL:', pnl)
avg_pnl = backtest_result[0].analyzers.getbyname(
    'myTradeAnalyzer').get_analysis()['pnl']['net']['average']
print('Avg PnL/Trade:', avg_pnl)

end_value = cerebro.broker.getvalue()

abs_rtn = (end_value - start_value)/start_value
print('Absolute Returns: {}%'.format(abs_rtn*100))

no_days = len(df.asfreq(freq='D').dropna())
cagr = (end_value/start_value)**(365/no_days) - 1
print('CAGR: {}%'.format(cagr*100))

print('Ending Portfolio Value: {}'.format(end_value))


------------------------------------------------
Starting Portfolio Value: 100000
Date: 2018-06-07T00:00:00, Signal: Short Position. Price: 7613.77734375
Date: 2018-06-08T00:00:00, Short Position Taken. Price: 7598.290966796865, Cost: -7598.290966796865, Commission: 7.598290966796865, Max Sell: 7978.2055151367085, Min Sell: 5318.803676757805, Position Size: -1
Date: 2018-06-08T00:00:00, Short Position Increase Signal. Price: 7561.166015625 Position Size: -1
Date: 2018-06-09T00:00:00, Short Position Taken. Price: 7546.878051757805, Cost: -1509.3756103515611, Commission: 1.5093756103515612, Max Sell: 7924.221954345696, Min Sell: 5282.814636230463, Position Size: -1.2
Date: 2018-06-09T00:00:00, Short Position Increase Signal. Price: 7415.0014648438 Position Size: -1.2
Date: 2018-06-10T00:00:00, Short Position Taken. Price: 7467.503222656244, Cost: -1493.500644531249, Commission: 1.4935006445312489, Max Sell: 7840.878383789057, Min Sell: 5227.252255859371, Position Size: -1.4
Date: 2018-06

![](assets/strat-1.png)
![](assets/strat-2.png)
![](assets/strat-3.png)

In [ ]:
cerebro.plot(iplot=False)
